In [58]:
# coding: utf-8
"""
igraphを使ったグラフクラスタリング
greedyアルゴリズム（louvain法）によってクラスタリングした後
サブグラフの中心性指標を図ることで単語の順位付けを行う
pagerankによるp(word|topic)の計算
purityによるスコアの計算
p(word|topic)から凸２次計画問題を解くことで、p(topic)を求める
サンプル文の抽出
PMIによってグラフクラスタリングする際の重みを変化させる
"""
from igraph import *
import csv
import collections
import pickle
import numpy as np
from openopt import QP

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def writecsv(arr, path):
    f = open(path, "ab")
    dataWriter = csv.writer(f)
    dataWriter.writerows(arr)
    f.close()

def readdump(path):
    f = open(path, "r")
    arr = pickle.load(f)
    f.close()
    return arr

# 有向エッジリストを入力して、重み付き無向ネットワークを出力する
# PMIを用いて重みを計算する
def cal_edgelist_to_network_PMI(list_edge):
    # 有向エッジリストを無向エッジリストに変換する
    list_edge = [tuple(sorted(row)) for row in list_edge]
    # ノードリスト
    dict_vertices = collections.Counter([word for row in list_edge for word in row])
    list_vertices, list_vertices_num = zip(*dict_vertices.items())
    # エッジリストとそのweightを作成
    tuple_edge, tuple_weight = zip(*collections.Counter(list_edge).items())
    # トータルのbigramの数
    total_bi = np.sum(tuple_weight)
    # トータルの単語数
    total_vo = np.sum(list_vertices_num)
    # weightの更新
    list_weight = []
    for row_edge, weight in zip(tuple_edge, tuple_weight):
        list_weight.append(np.log2(float(weight)/total_bi / (float(dict_vertices[row_edge[0]])/total_vo*float(dict_vertices[row_edge[1]])/total_vo)))
    return {"vertex": list(list_vertices), "edge": list(tuple_edge), "weight": list_weight}

# クラスタリング済みのネットワークを元にサブグラフのリストを作成
# vertexには全てのvertexを代入する（PageRankを計算するため）
def cal_cluster_to_network(dict_network):
    if dict_network.has_key("cluster") == False:
        print "クラスタリングができていません"
    
    # クラスタごとにwordをまとめる
    dict_cluster = collections.defaultdict(list)
    for word, cluster in zip(dict_network["vertex"], dict_network["cluster"]):
        dict_cluster[cluster].append(word)

    # リストに変換
    list_cluster_vertex = [row[1] for row in dict_cluster.items()]
    
    # 同様にエッジとウェイトのリストも作成する
    list_cluster_edge = []
    list_cluster_weight = []
    for cluster_vertex in list_cluster_vertex:
        list_cluster_edge_one = []
        list_cluster_weight_one = []
        # エッジリストの中に、一つでもノードが含まれていれば、そのクラスのノードに含める
        for row, weight in zip(dict_network["edge"], dict_network["weight"]):
            # and と or を切り替えることによって性能の比較
            if row[0] in cluster_vertex or row[1] in cluster_vertex:
                list_cluster_edge_one.append(row)
                list_cluster_weight_one.append(weight)
        list_cluster_edge.append(list_cluster_edge_one)
        list_cluster_weight.append(list_cluster_weight_one)
    
    # まとめる
    list_dict_network = [{"vertex": dict_network["vertex"],
                          "edge": cluster_edge,
                          "weight": cluster_weight}
                         for cluster_edge, cluster_weight
                         in zip(list_cluster_edge, list_cluster_weight)]
    
    return list_dict_network

# f_measureを計算する
def cal_f_measure(list_predict_measure):
    # 生成したクラスタ内のカウント
    dict_predict_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_predict_cluster[row[0]].append(row[1])
        
    # もとあるクラス内のカウント
    dict_measure_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_measure_cluster[row[1]].append(row[0])
    
    # local_purityの計算
    list_purity = []
    for row in dict_predict_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_purity.append([major_class, class_num])
    purity = float(np.sum(zip(*list_purity)[0])) / np.sum(zip(*list_purity)[1])
    print "Purity: ", purity
    
    # inverse_purityの計算
    list_inverse_purity = []
    for row in dict_measure_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_inverse_purity.append([major_class, class_num])
    inverse_purity = float(np.sum(zip(*list_inverse_purity)[0])) / np.sum(zip(*list_inverse_purity)[1])
    print "Inverse Purity: ", inverse_purity
    
    print "F-value: ", 2 / (1 / purity + 1 / inverse_purity)
    
# 凸２次計画問題を解いてp(topic)を求めるための関数
def cal_prob_topic(dict_network_master, list_dict_network_sub):
    prob_master = np.array([row[1] for row in sorted(zip(dict_network_master["vertex"], dict_network_master["page_rank"]), key=lambda x: x[0])])
    
    for i, dict_network_sub in enumerate(list_dict_network_sub):
        if i == 0:
            prob_sub = np.array([row[1] for row in sorted(zip(dict_network_sub["vertex"], dict_network_sub["page_rank"]), key=lambda x: x[0])])
        else:
            list_tmp = np.array([row[1] for row in sorted(zip(dict_network_sub["vertex"], dict_network_sub["page_rank"]), key=lambda x: x[0])])
            prob_sub = np.vstack((prob_sub, list_tmp))
    
    H = 2 * prob_sub.dot(prob_sub.T)
    f = -2 * prob_master.dot(prob_sub.T)
    Aeq = np.ones(len(list_dict_network_sub))
    beq = 1
    lb = np.zeros(len(list_dict_network_sub))
    
    p = QP(H, f, Aeq=Aeq, beq=beq, lb=lb)
    r = p.solve("cvxopt_qp")
    k_opt = r.xf
    return k_opt

###メイン部分
1. 有向エッジリストから無向重み付きエッジリストを作成
2. louvain法によるクラスタリング
3. クラスタごとにpagerankを計算し、p(word|topic)とする
4. クラスタごとにp(word|topic)を出力する辞書を作成

In [59]:
# エッジリストの読み込み
list_edge = readcsv("./files/kaigo_honne/list_edgelist20160127.csv")
# 元のネットワークを作成する（無向）
dict_network_master = cal_edgelist_to_network_PMI(list_edge)
# g = Graph(directed=True)
g_master = Graph()
g_master.add_vertices(dict_network_master["vertex"])
g_master.add_edges(dict_network_master["edge"])
# louvain法によるクラスタリング、vertexと同じ長さのクラスタ番号が書かれたリストがreturn
dict_network_master["cluster"] = g_master.community_fastgreedy(weights=dict_network_master["weight"]).as_clustering(n=7).membership
# 元のネットワークのpagerankを求める
dict_network_master["page_rank"] = g_master.pagerank(directed=False, weights=dict_network_master["weight"])
# クラスタ結果をもとにサブグラフのリストを作成
list_dict_network_sub = cal_cluster_to_network(dict_network_master)
# サブクラスタごとに中心性の計算
for i, dict_network_sub in enumerate(list_dict_network_sub):
    g_sub = Graph()
    g_sub.add_vertices(dict_network_sub["vertex"])
    g_sub.add_edges(dict_network_sub["edge"])
    list_dict_network_sub[i]["center_bet"] = g_sub.betweenness(directed=False, weights=dict_network_sub["weight"])
    list_dict_network_sub[i]["center_eigen"] = g_sub.eigenvector_centrality(directed=False, weights=dict_network_sub["weight"])
    list_dict_network_sub[i]["page_rank"] = g_sub.pagerank(directed=False, weights=dict_network_sub["weight"])
print "クラスタ数: ", len(list_dict_network_sub)

# トピックごとにwordを入力したらp(word|topic)が出るような辞書を作成
list_dict_prob = []
for i in range(len(list_dict_network_sub)):
    list_word_page = sorted(zip(list_dict_network_sub[i]["vertex"], list_dict_network_sub[i]["page_rank"]), key=lambda x: x[1], reverse=True)
    list_dict_prob.append({row[0]: row[1] for row in list_word_page})
    
# 凸２次計画問題を解いて、p(topic)を求める
list_prob_topic = cal_prob_topic(dict_network_master, list_dict_network_sub)

クラスタ数:  7

------------------------- OpenOpt 0.5625 -------------------------
problem: unnamed   type: QP
solver: cvxopt_qp
     pcost       dcost       gap    pres   dres
 0: -1.1509e-03 -1.0016e+00  1e+00  1e-16  3e+00
 1: -1.1516e-03 -1.1594e-02  1e-02  1e-16  3e-02
 2: -1.1995e-03 -1.6244e-03  4e-04  7e-17  1e-03
 3: -1.2920e-03 -1.3380e-03  5e-05  9e-17  1e-18
 4: -1.3023e-03 -1.3074e-03  5e-06  4e-17  1e-18
 5: -1.3028e-03 -1.3030e-03  2e-07  7e-17  8e-19
 6: -1.3028e-03 -1.3028e-03  3e-09  5e-17  1e-18
Optimal solution found.
istop: 1000 (optimal)
Solver:   Time Elapsed = 0.0 	CPU Time Elapsed = 0.0
objFuncValue: -0.001302825 (feasible, MaxResidual = 1.11022e-16)


In [66]:
num = 6
for row in sorted(list_dict_prob[num].items(), key=lambda x: x[1], reverse=True):
    print row[0], row[1]

足 0.0249356904487
喫茶 0.0233528351535
重視 0.0223387017979
仕方がない 0.0219882932928
寂しい 0.0197418067969
歓談 0.0182366991838
プロ 0.0173892102893
最適 0.0171153579712
ペット 0.0146103965854
爽やか 0.013194066421
折り紙 0.0117826275261
演奏家 0.0104702663816
定期 0.0102221740709
一緒に 0.00357342776682
感じ 0.0034350061739
コンサート 0.00310646187272
訪問 0.0029557616116
静か 0.00286719261276
強い 0.00273936733305
充実 0.00262800613519
開設 0.00258620682645
雰囲気 0.00255734123176
大勢 0.00255699355453
興味 0.00249996450678
映画鑑賞 0.00249584630621
場所 0.00249127004526
注文 0.00246825184229
健康診断 0.00244135065973
抜群 0.00242575724153
感 0.00240089380947
手 0.00238856496001
来訪 0.00238511767798
運行 0.00238162479781
魅力的 0.00236849030591
いろいろ 0.0023680286441
部分 0.00235873348944
訪問者 0.00234410592376
程度 0.00232568427569
ベッド 0.00231634367958
居住者 0.00230793099279
長い 0.00229471991074
入れ替わり 0.00227157769294
リハビリ 0.00225956426947
意味 0.00225901868156
一流 0.00225549123663
信頼 0.00225549123663
大丈夫 0.00224019322993
不自由 0.00223757662746
様々 0.00223670705199
店 0.002223

### 定量評価部分
1. 形態素解析済みセンテンスを読み込み、確率を計算し、どのクラスに属するか計算
2. 予測クラスと実際クラスのリストを作成する

In [67]:
list_sep_words = readdump("./files/kaigo_honne/list_sep_words_label.dump")
list_words= readcsv("./files/kaigo_honne/list_sentence_user_label.csv")
# 確率が最大になるクラスを予想する
# 実質ラベルがintじゃない場合は、エラーとして、記録しない
list_predict_measure = []
list_words_rev = []
error_count = 0
for row, row1 in zip(list_sep_words, list_words):
    try:
        class_tmp = 0
        prob_tmp = 0
        predict_class = []
        for num, dict_prob in enumerate(list_dict_prob):
            prob_tmp_tmp = 1
            for word in row[0]:
                prob_tmp_tmp *= float(dict_prob[word])
            prob_tmp_tmp *= list_prob_topic[num]
            if prob_tmp_tmp > prob_tmp:
                class_tmp = num
                prob_tmp = prob_tmp_tmp
        list_predict_measure.append([class_tmp, row[1]])
        list_words_rev.append(row1)
    except:
        error_count += 1
print "エラーデータ数: ", error_count

エラーデータ数:  68


###計算結果の表示

In [68]:
cal_f_measure(list_predict_measure)

Purity:  0.46279987853
Inverse Purity:  0.588824779836
F-value:  0.518261024817


In [69]:
cal_f_measure(list_predict_measure)

Purity:  0.46279987853
Inverse Purity:  0.588824779836
F-value:  0.518261024817
